# **Dataset Library**

In this chapter, you'll delve deeper into the capabilities of the 🤗 Datasets library. Here are some of the key questions you'll explore:

1. How to handle datasets not available on the Hugging Face Hub?
2. Techniques for slicing, dicing, and working with datasets, including using Pandas.
3. Handling large datasets that might overwhelm your system's RAM.
4. Understanding concepts like memory mapping and Apache Arrow.
5. Creating custom datasets and contributing them to the Hugging Face Hub.

Let's embark on this journey to enhance your understanding of 🤗 Datasets!

## What should I do if my dataset isn't available on the Hugging Face Hub?

You've learned how to utilize the Hugging Face Hub to fetch datasets, but there will be instances where you need to work with data stored locally on your laptop or on a remote server. In this section, we'll explore how 🤗 Datasets can be employed to load datasets that aren't accessible on the Hugging Face Hub.

### Working with local and remote datasets

🤗 Datasets simplifies the loading of local and remote datasets by providing loading scripts for various common data formats. Here are examples of loading scripts for different data formats:

- CSV & TSV: `load_dataset("csv", data_files="my_file.csv")`
- Text files: `load_dataset("text", data_files="my_file.txt")`
- JSON & JSON Lines: `load_dataset("json", data_files="my_file.jsonl")`
- Pickled DataFrames: `load_dataset("pandas", data_files="my_dataframe.pkl")`

The above illustrates that for each data format, specifying the type of loading script in the `load_dataset()` function is sufficient. Additionally, the `data_files` argument is used to provide the path to one or more files. Let's begin by loading a dataset from local files, and subsequently, we'll explore how to achieve the same with remote files.

### Loading a local dataset

For this example we’ll use the SQuAD-it dataset, which is a large-scale dataset for question answering in Italian.

The training and test splits are hosted on GitHub, so we can download them using the blow link:

https://github.com/crux82/squad-it/raw/master/SQuAD_it-train.json.gz

https://github.com/crux82/squad-it/raw/master/SQuAD_it-test.json.gz

Once you've downloaded them, unzip the files. You can see the compressed files has SQuAD_it-train.json and SQuAD_it-test.json, and that the data is stored in the JSON format.

Loading a JSON file using the `load_dataset()` function involves specifying whether the dataset is in standard JSON format (resembling a nested dictionary) or JSON Lines format (JSON separated by lines). In datasets like SQuAD-it, the information is stored in a nested structure, often with text contained within a specific field. To load this dataset correctly, we'd specify the `field` argument to accommodate this structure:

In [ ]:
from datasets import load_dataset

squad_it_dataset = load_dataset("json", data_files="SQuAD_it-train.json", field="data")

When loading datasets from local files, the default behavior is to generate a DatasetDict object containing at least one split, typically the train split. To verify this, you can inspect the `squad_it_dataset` object:

In [ ]:
squad_it_dataset

The output displays the count of rows along with the column names present in the training set. You can explore individual examples by selecting one from the train split.

In [ ]:
squad_it_dataset["train"][0]

That's the right approach! Having both the train and test splits within a single DatasetDict object allows for more efficient handling. Mapping each split name to its respective file using the data_files argument ensures the inclusion of both splits in a unified dataset object.

In [ ]:
data_files = {"train": "SQuAD_it-train.json", "test": "SQuAD_it-test.json"}
squad_it_dataset = load_dataset("json", data_files=data_files, field="data")
squad_it_dataset

Having both splits within a unified object facilitates uniform preprocessing across the entire dataset, ensuring consistency in the applied transformations or cleaning methods.

Datasets simplifies the process by handling file decompression automatically. Using compressed files directly in the `data_files` argument streamlines the loading process without the need for pre-decompression steps. This means we could have skipped the process of unzipping hte file manually by pointing the `data_files` argument directly to the compressed files.

In [ ]:
data_files = {"train": "SQuAD_it-train.json.gz", "test": "SQuAD_it-test.json.gz"}
squad_it_dataset = load_dataset("json", data_files=data_files, field="data")

Whether it's ZIP, TAR, or other common compression formats, 🤗 Datasets conveniently handles the decompression process upon loading, ensuring ease of use when working with compressed files directly in the `data_files` argument.

### Loading a remote dataset

When handling remote datasets, the process remains straightforward. Instead of directing the `data_files` argument to local paths, you simply assign it the URLs where the remote files are located. For instance, in the case of the SQuAD-it dataset residing on GitHub, the `data_files` parameter can point directly to the URLs hosting the SQuAD_it-*.json.gz files.

In [ ]:
url = "https://github.com/crux82/squad-it/raw/master/"
data_files = {
    "train": url + "SQuAD_it-train.json.gz",
    "test": url + "SQuAD_it-test.json.gz",
}
squad_it_dataset = load_dataset("json", data_files=data_files, field="data")

This retrieves the identical DatasetDict object we previously obtained, eliminating the need for manual downloading and decompression of the SQuAD_it-*.json.gz files. With this dataset at hand, let's delve into diverse data manipulation techniques!

### Toying with data subsets

In this section, we'll explore various techniques for slicing and dicing data using the 🤗 Datasets library. We'll cover operations like selecting specific columns, filtering rows based on conditions, and shuffling the dataset

Beyond Dataset.map(), 🤗 Datasets offers a range of methods to manage datasets. These functions empower you to filter rows, select columns, shuffle data, and more. Let's explore some of these to enhance our dataset manipulations.

In this instance, we'll work with the Drug Review Dataset available on the UC Irvine Machine Learning Repository. It includes patient reviews concerning different drugs, along with the treated condition and a 10-star rating reflecting patient satisfaction.

First we need to download and extract the data

https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip


In [1]:
!curl -O "https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip"
!tar -xvf drugsCom_raw.zip


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:03 --:--:--     0
100 94133    0 94133    0     0  22839      0 --:--:--  0:00:04 --:--:-- 22847
100  787k    0  787k    0     0   156k      0 --:--:--  0:00:05 --:--:--  156k
100 1823k    0 1823k    0     0   304k      0 --:--:--  0:00:05 --:--:--  304k
100 2875k    0 2875k    0     0   410k      0 --:--:--  0:00:07 --:--:--  525k
100 3807k    0 3807k    0     0   475k      0 --:--:--  0:00:08 --:--:--  764k
100 4747k    0 4747k    0     0   527k      0 --:--:--  0:00:08 --:--:--  954k
100 5455k    0 5455k    0     0   545k      0 --:--:--  0:00:09 --:--:--  938k
100 6223k    0 6223k    0     0   565k      0 --:--

TSV (Tab-Separated Values) functions similarly to CSV (Comma-Separated Values) but employs tabs instead of commas as the separator. To load TSV files, you can use the csv loading script and indicate the delimiter argument within the load_dataset() function, like this:

In [ ]:
from datasets import load_dataset

data_files = {"train": "drugsComTrain_raw.tsv", "test": "drugsComTest_raw.tsv"}
# \t is the tab character in Python
drug_dataset = load_dataset("csv", data_files=data_files, delimiter="\t")

One effective practice during data analysis is to extract a small random sample to gain a preliminary understanding of the data structure. In 🤗 Datasets, generating a random sample involves combining the `Dataset.shuffle()` and `Dataset.select()` functions in a sequence:

In [ ]:
drug_sample = drug_dataset["train"].shuffle(seed=42).select(range(1000))
# Peek at the first few examples
drug_sample[:3]